In [181]:
import keras

from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
#from keras.layers import CategoryEncoding
import keras_tuner as kt


import os

import tensorflow as tf

import chess
import chess.pgn


import numpy as np

#initialise data to be empty to see if we need to generate it
X = None
y1 = None
y2 = None

In [ ]:
def get_ratings(game):

    return int(game.headers['WhiteElo']),int(game.headers['BlackElo'])

def rating_to_output(rating):
    ret = np.zeros(48)
    r = int((rating-600)/50)
    if r>47:
        r = 47
    if r<0:
        r = 0
    ret[r] = 1
    return ret

def get_game_tensors(game_pgn,num_tensors):

    gt = np.zeros((num_tensors,134))

    moves = []
    evals = []

    for m in game_pgn.mainline():
        moves.append(m.san())
        if m.eval() is None: #should only happen on a mate
            evals.append(None)
        else:
            evals.append(m.eval().white() if m.turn() == chess.WHITE else m.eval().black())

    #let our t vector be a 1D array of 133 elements. The first 128 element represent the board before the move is made and after the move is made. The 129th element is the evaluation of the move before it is made and the 130th element is the evaluation of the move after it is made. If it is mate in X moves before the move is made, the 131st element is 1 and 0 otherwise. If it is mate in X moves after the move is made, the 132nd element is 1 and 0 otherwise. The 133rd element is 1 if it is white moved and -1 if it is black making the move.

    board = chess.Board()

    for m in range(0,min(40,len(moves)-1)):
        t = np.zeros(134)

        for i in range(64): #original board position
            if board.piece_at(i) is not None:
                                t[i] = board.piece_at(i).piece_type * (1 if board.piece_at(i).color == chess.WHITE else -1)

        board.push_san(moves[m])
        for i in range(64):
            if board.piece_at(i) is not None:
                t[i+64] = board.piece_at(i).piece_type * (1 if board.piece_at(i).color == chess.WHITE else -1)

        #evals is either a number, or starts with a #. If it starts with a #, it is a mate in X moves

        #handle the case of a mate
        if evals[m] is None:
            t[129] = 0
            t[130] = 1
        if evals[m+1] is None:
            t[131] = 0
            t[132] = 1

        if evals[m] is not None and evals[m].is_mate():
            t[129] = float(evals[m].mate())
            t[130] = 1
        elif evals[m] is not None:
            t[129] = float(evals[m].score()/100)
            t[130] = 0

        if evals[m+1] is not None and evals[m+1].is_mate():
            t[131] = float(evals[m+1].mate())
            t[132] = 1
        elif evals[m+1] is not None:
            t[131] = float(evals[m+1].score()/100)
            t[132] = 0

        t[133] = -1 if board.turn == chess.WHITE else 1

        gt[m] = t

    return gt

In [ ]:
def make_data(path,target_file):
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    X,y1,y2 = [],[],[]
    for f in files:
        with open(os.path.join(path,f)) as pgn:
            game = chess.pgn.read_game(pgn)
            gt = get_game_tensors(game,40)
            y1t = get_ratings(game)[0]
            y2t = get_ratings(game)[1]

            X.append(gt)
            y1.append(y1t)
            y2.append(y2t)

    X = np.array(X)
    y1 = np.array(y1)
    y2 = np.array(y2)

    #save the data to the target file
    np.save(os.path.join(path,target_file + "_X.npy",X))
    np.save(os.path.join(path,target_file + "_y1.npy",y1))
    np.save(os.path.join(path,target_file + "_y2.npy",y2))

def load_data(path,target_file):
    X = np.load(os.path.join(path,target_file + "_X.npy"))
    y1 = np.load(os.path.join(path,target_file + "_y1.npy"))
    y2 = np.load(os.path.join(path,target_file + "_y2.npy"))

    return X,y1,y2

In [ ]:
def model_builder(hp):

    lstm1_units = hp.Int('lstm1_units',min_value=32,max_value=512,step=32)
    lstm2_units = hp.Int('lstm2_units',min_value=32,max_value=512,step=32)
    dense1_units = hp.Int('dense1_units',min_value=32,max_value=512,step=32)
    dense2_units = hp.Int('dense2_units',min_value=32,max_value=512,step=32)
    input = Input(shape=(40, 134))
    x = LSTM(lstm1_units,return_sequences=True)(input)
    x = LSTM(lstm2_units)(x)
    x = Dense(dense1_units,activation='relu')(x)
    x = Dense(dense2_units,activation='relu')(x)

    output1 = Dense(1,activation='relu',name="WhiteElo")(x)
    output2 = Dense(1,activation='relu',name="BlackElo")(x)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model = keras.Model(inputs=input,outputs=[output1,output2])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                    loss={'WhiteElo':'mse','BlackElo':'mse'},
                    metrics={'WhiteElo':'mae','BlackElo':'mae'})

    return model

In [ ]:
#if the data doesn't exist, generate it
if not os.path.exists("data/all_data/data_X.npy"):
    make_data("data/all_data","data")
X,y1,y2 = load_data("data/all_data","data")

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                    max_epochs=100,
                    factor=3)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
save = tf.keras.callbacks.ModelCheckpoint('model.keras', save_best_only=True,mode='auto',monitor='val_loss')

tuner.search(X,(y1,y2),epochs=100,validation_split=0.2,callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.values)

model = tuner.hypermodel.build(best_hps)

history = model.fit(X,(y1,y2),epochs=100,validation_split=0.2,callbacks=[stop_early,save])
model.save('model3.keras')